<a href="https://colab.research.google.com/github/akaraci/COVID-19/blob/master/VGGCOV19_NET_and_VGG19_MultiClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autotime
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn import metrics
from google.colab import drive
drive.mount('/content/drive')


import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.callbacks import EarlyStopping
np.random.seed(1000)

from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import  ImageDataGenerator, img_to_array, load_img
from glob import glob

In [ ]:
#------------VGGCOV19-NET  Multiclass Code 

from sklearn.preprocessing import label_binarize
from itertools import cycle
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 3)


vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer  canceled
vgg_layer_list=vgg.layers

model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)


for layer in model.layers:
    layer.trainable=False

numberOfClass=3
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(256))
model.add(Dense(512))

model.add(Dense(numberOfClass,activation='softmax'))


from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)
aucs = []
tprs = []
fprs = []
base_fpr = np.linspace(0, 1, 101)
history=[]
acc_val=[]
result=[]
plt.figure(figsize=(10, 10))
fold=1
opt=keras.optimizers.Adam(lr=0.001)
for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    history = model.fit(x_train, y_train, batch_size=32, epochs=100, verbose=0) #validation_split=0.2   validation_data=(x_test, y_test)
    
    np.save("drive/My Drive/Datacovid_kemal/modelsmodify/my_historyxxx"+str(fold)+".npy",history.history)
    
    result.append(model.predict(x_test))
    prediction = model.predict(x_test)
    Y_pred_classes = np.argmax(prediction,axis = 1) 
    y_test_classes = np.argmax(y_test,axis = 1)
    print(metrics.classification_report(y_test_classes, Y_pred_classes, digits=3))
    conf_mat =confusion_matrix(y_test_classes, Y_pred_classes, labels=[0, 1, 2])
    print(conf_mat)
    total1=sum(sum(conf_mat))
    accuracy1=(conf_mat[0,0]+conf_mat[1,1]+conf_mat[2,2])/total1
    acc_val.append(accuracy1)
    print ('Accuracy : ', accuracy1)
    y_true1 = label_binarize(y_test_classes, classes=[0, 1, 2])
    y_pred1 = label_binarize(Y_pred_classes, classes=[0, 1, 2])
    n_classes = y_true1.shape[1]
  
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ =  metrics.roc_curve(y_true1[:, i], y_pred1[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    print("Roclar", roc_auc[0],roc_auc[1],roc_auc[2])
    
    fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y_true1.ravel(), y_pred1.ravel())
    roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])
    fprs.append(all_fpr)
    tprs.append(mean_tpr)
    aucs.append(roc_auc["macro"])
    plt.plot(fpr["macro"], tpr["macro"],
            label='ROC Fold- %d (AUC = %0.2f)' % (fold, roc_auc["macro"]), lw=2, alpha=0.3)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    lw = 2
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve for each fold')
    plt.legend(loc="lower right")
    fold=fold+1
print("Mean ACC=",np.mean(acc_val))
mean_auc=np.mean(aucs)
print("Mean AUC:",mean_auc)
tprs = np.array(tprs)
plt.text(0.7,0.2,'Mean ROC (AUC = %0.2f)' % (mean_auc), color='black', fontsize=11)
plt.show()
learn.recorder.plot_metrics()

In [ ]:
##------VGG19 Multiclass Code 
from sklearn.preprocessing import label_binarize
from itertools import cycle
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 3)
vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer canceled
#print(vgg.summary())
vgg_layer_list=vgg.layers
#print(vgg_layer_list)
model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)
#print(model.summary())

for layer in model.layers:
    layer.trainable=False

numberOfClass=3
model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(4096))
model.add(Dense(3,activation="softmax"))

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)
aucs = []
tprs = []
fprs = []
base_fpr = np.linspace(0, 1, 101)
history=[]
acc_val=[]
result=[]
plt.figure(figsize=(10, 10))
fold=1
opt=keras.optimizers.Adam(lr=0.001)
for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    history = model.fit(x_train, y_train, batch_size=32, epochs=100, verbose=0) #validation_split=0.2   validation_data=(x_test, y_test)
    
    np.save("drive/My Drive/Datacovid_kemal/modelsmodify/my_historyvggxxx"+str(fold)+".npy",history.history)
    
    result.append(model.predict(x_test))
    prediction = model.predict(x_test)
    Y_pred_classes = np.argmax(prediction,axis = 1) 
    y_test_classes = np.argmax(y_test,axis = 1)
    #print(metrics.confusion_matrix(y_test_classes, Y_pred_classes))
    print(metrics.classification_report(y_test_classes, Y_pred_classes, digits=3))
    conf_mat =confusion_matrix(y_test_classes, Y_pred_classes, labels=[0, 1, 2])
    print(conf_mat)
    total1=sum(sum(conf_mat))
    accuracy1=(conf_mat[0,0]+conf_mat[1,1]+conf_mat[2,2])/total1
    acc_val.append(accuracy1)
    print ('Accuracy : ', accuracy1)
    #probas_ = model.predict_proba(x_test)
     
    y_true1 = label_binarize(y_test_classes, classes=[0, 1, 2])
    y_pred1 = label_binarize(Y_pred_classes, classes=[0, 1, 2])
    n_classes = y_true1.shape[1]
  
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ =  metrics.roc_curve(y_true1[:, i], y_pred1[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    print("Roclar", roc_auc[0],roc_auc[1],roc_auc[2])
    
    fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y_true1.ravel(), y_pred1.ravel())
    roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])
    fprs.append(all_fpr)
    tprs.append(mean_tpr)
    aucs.append(roc_auc["macro"])
    plt.plot(fpr["macro"], tpr["macro"],
            label='ROC Fold- %d (AUC = %0.2f)' % (fold, roc_auc["macro"]), lw=2, alpha=0.3)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    lw = 2
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve for each fold')
    plt.legend(loc="lower right")
    fold=fold+1
print("Mean ACC=",np.mean(acc_val))
mean_auc=np.mean(aucs)
print("Mean AUC:",mean_auc)
tprs = np.array(tprs)
plt.text(0.7,0.2,'Mean ROC (AUC = %0.2f)' % (mean_auc), color='black', fontsize=11)
plt.show()
learn.recorder.plot_metrics()
    


In [ ]:
#VGGCOV19-NET Summary
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 3)


vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer  canceled
#print(vgg.summary())
vgg_layer_list=vgg.layers
#print(vgg_layer_list)
model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)
#print(model.summary())

for layer in model.layers:
    layer.trainable=False

numberOfClass=3
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(256))
model.add(Dense(512))

model.add(Dense(numberOfClass,activation='softmax'))
print(model.summary())

In [ ]:
#VGG19-NET Summary
X = pickle.load(open("drive/My Drive/Datacovid_kemal/X_tum.pickle", "rb"))
y = pickle.load(open("drive/My Drive/Datacovid_kemal/Y_tum.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
y = to_categorical(y, num_classes = 3)
vgg=VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))   #include_top=False fully connected layer canceled
#print(vgg.summary())
vgg_layer_list=vgg.layers
#print(vgg_layer_list)
model=Sequential()
for layer in vgg_layer_list:
    model.add(layer)
#print(model.summary())

for layer in model.layers:
    layer.trainable=False

numberOfClass=3
model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(4096))
model.add(Dense(3,activation="softmax"))
print(model.summary())

In [ ]:
pip install scikit-plot

In [ ]:
!pip install ipython-autotime